In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.utils.prune as prune
import torch.nn.functional as F
import random
import pandas as pd
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
#training data aaa
train_data = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)

#test data
test_data = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

testloader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False, num_workers=2)


# from google.colab import drive
# drive.mount('/content/gdrive')

/home/farzin/anaconda3/envs/hp_dnn/lib/python3.6/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Files already downloaded and verified
Files already downloaded and verified


In [2]:
model_save_name = 'VGG11_80.pt'
path = F"./model/VGG11/{model_save_name}"
AlexNet_model = torchvision.models.vgg11_bn(pretrained=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = AlexNet_model.to(device)
model = torch.load(path, map_location=device)

model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(ke

In [ ]:
pip install pytorch-ignite

In [3]:
from ignite.metrics import Precision,Recall,Accuracy,ConfusionMatrix,TopKCategoricalAccuracy
precision = Precision(device=device)
#confusionMatrix=ConfusionMatrix(10,device=device)
recall=Recall(device=device)
acc=Accuracy(device=device)
T_acc=TopKCategoricalAccuracy(k=5,device=device)
criterion = torch.nn.CrossEntropyLoss()

sm = torch.nn.Softmax(dim=1)
def test(model):
    confi=0
    sub_confi=0
    correct=0
    total=0
    with torch.no_grad():

        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            #loss = criterion(outputs, labels)
            precision.update((outputs, labels))
            #confusionMatrix.update((outputs, labels))
            recall.update((outputs, labels))
            acc.update((outputs, labels))
            T_acc.update((outputs, labels))
            probabilities = sm(outputs)
            topk=torch.topk(probabilities, 1)
            topk2=torch.topk(probabilities, 2)
            cols = torch.chunk(topk2.values, 2, 1)
            sub_confi+=(cols[0].sum()-cols[1].sum())/32
            confi+=topk.values.sum()/32
            index_conf=torch.nonzero(topk.values.reshape(-1)>0.50)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted[index_conf] == labels[index_conf]).sum()
            #print(topk.values)
            total += labels.size(0)

        return_acc=acc.compute()
        return_pre=precision.compute()
        return_rec=recall.compute()
        return_tacc= T_acc.compute()
        return_con=confi/len(testloader)
        return_sub_con=sub_confi/len(testloader)
        precision.reset()
        recall.reset()
        acc.reset()
        #confusionMatrix.reset()
        T_acc.reset()
        acc_50=correct/total
        return return_acc,return_pre,return_rec,return_tacc,return_con.item(),return_sub_con.item(),acc_50.item()


In [7]:
test(model)

(0.7475,
 tensor([0.9495, 0.8333, 0.7765, 0.6778, 0.4914, 0.7059, 0.8222, 0.7766, 0.9293,
         0.8725, 0.7126, 0.5050, 0.8155, 0.7895, 0.7184, 0.7009, 0.7358, 0.8763,
         0.7935, 0.7528, 0.9091, 0.7963, 0.7545, 0.8421, 0.7983, 0.6495, 0.7889,
         0.5736, 0.8081, 0.6600, 0.7018, 0.7363, 0.7030, 0.6311, 0.7778, 0.4779,
         0.7778, 0.7767, 0.6476, 0.8571, 0.7188, 0.8529, 0.7170, 0.7938, 0.7093,
         0.6778, 0.5865, 0.6593, 0.8879, 0.7895, 0.5978, 0.8283, 0.5390, 0.8654,
         0.7890, 0.5366, 0.8317, 0.8019, 0.8788, 0.7250, 0.8241, 0.8090, 0.6944,
         0.6923, 0.6824, 0.6304, 0.7653, 0.6552, 0.8932, 0.8763, 0.7453, 0.7664,
         0.4949, 0.6636, 0.5091, 0.8393, 0.8598, 0.7692, 0.7526, 0.7864, 0.6979,
         0.7917, 0.8911, 0.7579, 0.7551, 0.8700, 0.8588, 0.8173, 0.8191, 0.8544,
         0.7383, 0.8095, 0.7386, 0.6500, 0.8879, 0.6887, 0.7160, 0.7670, 0.5579,
         0.8191], device='cuda:0', dtype=torch.float64),
 tensor([0.9400, 0.8500, 0.6600, 0.6100, 0.

In [4]:
# ================================================================
# change number to bit representation
# ================================================================
import numpy as np

def IEEE754_v2_tensor(numbers):

    signs = np.where(numbers < 0, 1, 0)
    numbers = np.abs(numbers)

    int_parts = np.floor(numbers).astype(int)
    dec_parts = numbers - int_parts

    int_bin_parts = np.array([bin(x).replace('0b', '') if x > 0 else '' for x in int_parts])

    mantissas = []
    exponents = []

    for i in range(len(numbers)):
        if int_parts[i] > 0:
            mantissa = int_bin_parts[i][1:] + fractional_to_bin(dec_parts[i], 23 - len(int_bin_parts[i][1:]))
            exponent = len(int_bin_parts[i]) - 1
        else:
            fraction_bin = fractional_to_bin(dec_parts[i], 50)
            first_one = fraction_bin.find('1')
            exponent = -(first_one + 1)
            mantissa = fraction_bin[first_one + 1:first_one + 24]

        mantissa = (mantissa + '0' * 23)[:23]
        mantissas.append(mantissa)
        exponents.append(exponent)

    exponents = np.array(exponents) + 127
    exponent_bits = np.array([bin(e).replace('0b', '').zfill(8) for e in exponents])

    ieee754_representations = np.array([
        str(signs[i]) + exponent_bits[i] + mantissas[i] for i in range(len(numbers))
    ])

    return ieee754_representations


def fractional_to_bin(dec_part, length=24):

    mantissa = ''
    for _ in range(length):
        dec_part *= 2
        int_part = int(dec_part)
        mantissa += str(int_part)
        dec_part -= int_part
        if dec_part == 0:
            break
    return mantissa + '0' * (length - len(mantissa))  # Pad to ensure fixed length

import numpy as np

def inv_IEEE754_tensor(num_IEEE_array):

    binary_matrix = np.array([list(num) for num in num_IEEE_array], dtype=int)


    signs = binary_matrix[:, 0]

    exponent_bits = binary_matrix[:, 1:9]
    exponents = np.dot(exponent_bits, 2 ** np.arange(7, -1, -1))
    mantissa_bits = binary_matrix[:, 9:].astype(float)
    powers = 2.0 ** np.arange(-1, -mantissa_bits.shape[1] - 1, -1, dtype=float)
    mantissas = np.dot(mantissa_bits, powers)
    normalized_mantissas = 1.0 + mantissas
    is_subnormal = (exponents == 0)
    exponents = np.where(is_subnormal, -126, exponents - 127)
    mantissas = np.where(is_subnormal, mantissas, normalized_mantissas)
    is_zero = (exponents == -127) & (mantissa_bits.sum(axis=1) == 0)
    numbers = mantissas * (2.0 ** exponents)
    numbers = np.where(is_zero, 0.0, numbers)
    numbers = np.where(signs == 1, -numbers, numbers)

    return numbers



In [5]:

import numpy as np
import torch

def bitFLIP_v3_tensor(original_values, positions_list):
    original_values_np = original_values.cpu().detach().numpy()  # Convert Torch tensor to NumPy array

    ieee_binary_strings = IEEE754_v2_tensor(original_values_np)  # Convert to IEEE 754 binary
    flipped_binaries = []
    for i, positions in enumerate(positions_list):
        str_num = list(ieee_binary_strings[i])
        for position in positions:
            bit_position = 31 - position  # Convert to IEEE754 bit position
            if bit_position == 1:  # Prevent flipping sign bit
                bit_position = 0
            #print(len(str_num))
            str_num[bit_position] = '0' if str_num[bit_position] == '1' else '1'
        if(original_values_np[i]==0):
          #print(original_values_np[i])
          str_num='00000000000000000000000000000000'
        flipped_binaries.append("".join(str_num))

    flipped_values = inv_IEEE754_tensor(np.array(flipped_binaries))

    flipped_values_tensor = torch.tensor(flipped_values, dtype=original_values.dtype, device=original_values.device, requires_grad=True)

    return flipped_values_tensor




In [4]:
import torch
import random
import pandas as pd
from collections import Counter

def fault_tolerance_two_agree(BER, model):

    t = torch.cat([param.view(-1) for name, param in model.named_parameters()
                   if "weight" in name and "norm" not in name]).to('cuda:0')

    count = len(t)
    nums = int(count * 31 * BER)
    print(f"Total number of weights (float32): {count}")

# yani agar num = 6 bashe va count 20, ma az beyn 0 ta 620 byd 6 ta random pos entekhab konim
# in this scenario count is 129167744 and the reason why is explained below
    def random_bit_positions():
        return random.sample(range(0, 31 * count), nums)
    

    pos1 = random_bit_positions()
    pos2 = random_bit_positions()
    pos3 = random_bit_positions()

    # Step 3: Count occurrences across all three selections
    all_positions = pos1 + pos2 + pos3
    pos_counts = Counter(all_positions)

    # Step 4: Keep positions that appear at least twice
    two_or_more = [pos for pos, cnt in pos_counts.items() if cnt >= 2]

    if len(two_or_more) == 0:
        print("No 2-agreement bit flips found. Nothing to do.")
        return

    lst_sorted_final = torch.tensor(sorted(two_or_more), device='cuda:0')


    bit_positions = lst_sorted_final % 31
    index_positions = (lst_sorted_final - bit_positions) // 31


    bits_grouped = pd.DataFrame({
        'index': index_positions.cpu(),
        'bit': bit_positions.cpu()
    }).groupby('index', sort=False)['bit'].apply(list).to_dict()

    unique_indices = torch.tensor(list(bits_grouped.keys()), device='cuda:0')
    bit_flips = [torch.tensor(bits_grouped[idx.item()], device='cuda:0') for idx in unique_indices]


    mask = (t[unique_indices] != 0)
    keep_idx = mask.nonzero(as_tuple=False).flatten()

    if len(keep_idx) == 0:
        print("No non-zero values to flip. Exiting.")
        return

    unique_indices = unique_indices[keep_idx]
    bit_flips = [bit_flips[i] for i in keep_idx.tolist()]

    flipped_values = bitFLIP_v3_tensor(t[unique_indices], bit_flips)


    start = 0
    for name, param in model.named_parameters():
        if "weight" in name and "norm" not in name:
            param_size = param.numel()
            end = start + param_size
            mask = (unique_indices >= start) & (unique_indices < end)

            if mask.any():
                update_indices = unique_indices[mask] - start
                param_flat = param.view(-1).clone()

                non_zero_mask = param_flat[update_indices] != 0
                if non_zero_mask.any():
                    valid_update_indices = update_indices[non_zero_mask]
                    valid_flipped_values = flipped_values[mask][non_zero_mask]
                    param_flat[valid_update_indices] = valid_flipped_values

                param.data.copy_(param_flat.view(param.shape))

            start = end


In [7]:
def count_eligible_weights(model):
    # Flatten and concatenate all eligible weight parameters
    t = torch.cat([param.view(-1) for name, param in model.named_parameters()
                   if "weight" in name and "norm" not in name])
    count = len(t)
    print(f"Total number of weights (float32): {count}")
    return count
count_eligible_weights(model)

Total number of weights (float32): 129167744


129167744

Conv Layers      ≈     9,217,728
BatchNorm Layers ≈         2,752
FC Layers        ≈   119,947,264
-------------------------------
Total            ≈   129,167,744
Total for each block:

Conv(3→64): 64×3×3×3 = 1,728

Conv(64→128): 128×64×3×3 = 73,728

Conv(128→256): 256×128×3×3 = 294,912

Conv(256→256): 256×256×3×3 = 589,824

Conv(256→512): 512×256×3×3 = 1,179,648

Conv(512→512): 512×512×3×3 = 2,359,296

Conv(512→512): same = 2,359,296

Conv(512→512): same = 2,359,296

Total Conv Weights: ≈ 9,217,728

BatchNorm Layers:
Each BatchNorm2d(N) has:

weight and bias: 2 × N

So total:

64 + 128 + 256×2 + 512×4 = 64 + 128 + 512 + 2048 = 2752 parameters

Fully Connected Layers:
Linear(25088 → 4096): 25088×4096 = 102,760,448

Linear(4096 → 4096): 4096×4096 = 16,777,216

Linear(4096 → 100): 4096×100 = 409,600

In [8]:
#doual
Accuracy=[]
Precision=[]
Recall=[]
Tacc=[]
conf=[]
sub_conf=[]
fault_position_array=[]
bits_array=[]
acc_50=[]
M=6
power=-3
while (power<-1):
  for i in range (50):
    print(power)
    BER=5*(10**power)
    #fault_position,bits=fault_positions(model,BER)
    fault_tolerance_two_agree( BER, model)
    return_acc,return_pre,return_rec,return_tacc,return_conf,return_sub_conf,return_acc_50=test(model)
    Accuracy.append(return_acc)
    Precision.append(return_pre)
    Recall.append(return_rec)
    Tacc.append(return_tacc)
    conf.append(return_conf)
    sub_conf.append(return_sub_conf)
    acc_50.append(return_acc_50)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Use GPU if available
    model = torch.load(path, weights_only=False)
    model.eval()
    print(Accuracy)
  power+=1

-3
Total number of weights (float32): 129167744


/home/farzin/anaconda3/envs/hp_dnn/lib/python3.6/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /tmp/pip-req-build-__fqk7_8/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


TypeError: 'weights_only' is an invalid keyword argument for this function

['00000000000000000000000000000000', '10111110001000000000000000000000']